##Classifying 4500 unlabeled text with weak labels

In [ ]:
!wget https://raw.githubusercontent.com/DagimB/6188_Recipe_Text_Classification/main/data/unlabeled_train.jsonl

--2024-03-16 03:06:20--  https://raw.githubusercontent.com/DagimB/6188_Recipe_Text_Classification/main/data/unlabeled_train.jsonl
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1200786 (1.1M) [text/plain]
Saving to: ‘unlabeled_train.jsonl’

unlabeled_train.jso 100%[===================>]   1.14M  --.-KB/s    in 0.05s   

2024-03-16 03:06:21 (21.0 MB/s) - ‘unlabeled_train.jsonl’ saved [1200786/1200786]



In [ ]:
!git clone https://github.com/DagimB/6188_Recipe_Text_Classification.git

Cloning into '6188_Recipe_Text_Classification'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 31 (delta 6), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (31/31), 1.08 MiB | 4.30 MiB/s, done.
Resolving deltas: 100% (6/6), done.


In [ ]:
# Path to the spaCy model within the cloned repository
model_path = "/content/6188_Recipe_Text_Classification/output/experiment-1/model-best"

In [ ]:
import spacy
import json
from pathlib import Path

def score_records(input_file, model_path, threshold):
    nlp = spacy.load(model_path)

    processed_records = []

    with open(input_file, 'r', encoding='utf-8') as f:
        for line in f:
            record = json.loads(line)

            doc = nlp(record['text'])

            cats = {label: score for label, score in doc.cats.items()}

            accept = [label for label, score in cats.items() if score >= threshold]

            options = [{'id': label, 'text': label, 'meta': f'{score:.2f}'} for label, score in cats.items()]


            processed_record = {
                'text': record['text'],
                'cats': cats,
                'accept': accept,
                'answer': 'accept' if accept else 'reject',
                'options': options
            }


            processed_records.append(processed_record)


    output_file = input_file.replace('.jsonl', '_processed.jsonl')
    with open(output_file, 'w', encoding='utf-8') as f:
        for record in processed_records:
            f.write(json.dumps(record, ensure_ascii=False) + '\n')

input_file = 'unlabeled_train.jsonl'
threshold = 0.5
score_records(input_file, model_path, threshold)
